In [1]:
from sas7bdat import SAS7BDAT
import pandas as pd
from IPython.display import display
import gc
import pyreadstat

In [2]:
# kyrbs_file_path = 'c:\\data\\education\\kyrbs2024_sas\\kyrbs\\kyrbs2024.sas7bdat'
# pop_file_path = 'c:\\data\\education\\kyrbs2024_sas\\kyrbs\\pop24.sas7bdat'

# enc = 'euc-kr'
# df1, meta1 = pyreadstat.read_sas7bdat(kyrbs_file_path, encoding=enc)
# df2, meta2 = pyreadstat.read_sas7bdat(pop_file_path, encoding=enc)

# display(df1.head())
# display(df2.head())

# df1.to_csv('c:\\data\\education\\kyrbs2024_sas\\kyrbs\\kyrbs2024.csv', encoding='cp949', index=False)
# df2.to_csv('c:\\data\\education\\kyrbs2024_sas\\kyrbs\\pop24.csv', encoding='cp949', index=False)

# del df1
# del df2
# gc.collect()

In [3]:
kyrbs = pd.read_csv('c:\\data\\education\\kyrbs2024_sas\\kyrbs\\kyrbs2024.csv', encoding='cp949')
pop = pd.read_csv('c:\\data\\education\\kyrbs2024_sas\\kyrbs\\pop24.csv', encoding='cp949')

display(kyrbs.head(3))
display(pop.head(3))

,OBS,mod_d,YEAR,CITY,CTYPE,CTYPE_SD,MH,SCHOOL,STYPE,STRATA,...,E_LT_F,E_LT_SF,E_LT_M,E_LT_SM,E_EDU_F,E_KRN_F,E_BORN_F,E_EDU_M,E_KRN_M,E_BORN_M
0,A100001,2024.11.26.,2024.0,서울,대도시,대도시,중학교,중학교,남녀공학,2024_64,...,1.0,9999.0,1.0,9999.0,3.0,1.0,9999.0,3.0,1.0,9999.0
1,A100002,2024.11.26.,2024.0,서울,대도시,대도시,중학교,중학교,남녀공학,2024_64,...,1.0,9999.0,1.0,9999.0,3.0,1.0,9999.0,3.0,1.0,9999.0
2,A100003,2024.11.26.,2024.0,서울,대도시,대도시,중학교,중학교,남녀공학,2024_64,...,1.0,9999.0,1.0,9999.0,3.0,1.0,9999.0,3.0,1.0,9999.0


,db_y,_total_,STRATA
0,2024.0,43.0,2024_1
1,2024.0,57.0,2024_10
2,2024.0,35.0,2024_100


In [4]:
missing_cols = kyrbs.columns[kyrbs.isna().any()].tolist()
print(missing_cols)
display(kyrbs[missing_cols].isna().sum())

['SCHOOL', 'PA_SWD_S', 'PA_SWD_N', 'PA_SWK_S', 'PA_SWK_N', 'HT', 'WT', 'M_SLP_HR', 'M_SLP_MM', 'M_WK_HR', 'M_WK_MM', 'AC_FAGE', 'TC_FAGE', 'TC_DAGE', 'TC_EC_FAGE', 'TC_EC_DAGE', 'TC_HTP_FAGE', 'TC_HTP_DAGE', 'S_FAGE', 'INT_SPWD_TM', 'INT_SPWK_TM', 'AGE', 'AGE_M', 'E_S_RCRD', 'E_SES', 'E_RES']


SCHOOL          441
PA_SWD_S        967
PA_SWD_N        967
PA_SWK_S       1004
PA_SWK_N       1004
HT             1480
WT             1480
M_SLP_HR       4140
M_SLP_MM       4140
M_WK_HR        4140
M_WK_MM        4140
AC_FAGE          59
TC_FAGE          87
TC_DAGE          32
TC_EC_FAGE       30
TC_EC_DAGE       23
TC_HTP_FAGE      32
TC_HTP_DAGE      15
S_FAGE          155
INT_SPWD_TM    1821
INT_SPWK_TM    1855
AGE              49
AGE_M            49
E_S_RCRD          3
E_SES             5
E_RES             7
dtype: int64

In [5]:
kyrbs['disturbed'] = kyrbs[[f'M_GAD_{i}' for i in range(1, 8)]].apply(lambda x: 2 if (x >= 3).any() else 1, axis=1) # 어떤것에 자주 방해 받는지
kyrbs['disturbed'].value_counts()

disturbed
1    35952
2    18701
Name: count, dtype: int64

In [6]:
kyrbs['violent'] = kyrbs['V_TRT'].apply(lambda x: 1 if x == 1 else (3 if x == 7 else 2)) # 폭력 경험 - 1 없음, 2 있음, 3 자주(6회 이상)
kyrbs['violent'].value_counts()

violent
1    53249
2     1246
3      158
Name: count, dtype: int64

In [7]:
display(kyrbs['E_FM_F_1'].value_counts())
display(kyrbs['E_FM_SF_2'].value_counts())

E_FM_F_1
1.0       46840
8888.0     5917
9999.0     1896
Name: count, dtype: int64

E_FM_SF_2
9999.0    47004
8888.0     5917
2.0        1732
Name: count, dtype: int64

In [8]:
# 가족관련이 선택질문인데 8888은 무응답이므로 제외함
kyrbs = kyrbs[kyrbs['E_FM_F_1'] != 8888]

In [9]:
display(kyrbs['E_FM_F_1'].value_counts())
display(kyrbs['E_FM_SF_2'].value_counts())
display(kyrbs[(kyrbs['E_FM_F_1'] == 9999) & (kyrbs['E_FM_SF_2'] == 9999)].shape[0])

E_FM_F_1
1.0       46840
9999.0     1896
Name: count, dtype: int64

E_FM_SF_2
9999.0    47004
2.0        1732
Name: count, dtype: int64

1610

In [10]:
display(kyrbs['E_FM_M_3'].value_counts())
display(kyrbs['E_FM_SM_4'].value_counts())

E_FM_M_3
3.0       47405
9999.0     1331
Name: count, dtype: int64

E_FM_SM_4
9999.0    47258
4.0        1478
Name: count, dtype: int64

In [11]:
print("father")
print(kyrbs[(kyrbs['E_FM_F_1'] == 1) & (kyrbs['E_FM_SF_2'] == 9999)].shape[0])
print(kyrbs[(kyrbs['E_FM_F_1'] == 9999) & (kyrbs['E_FM_SF_2'] == 2)].shape[0])
print(kyrbs[(kyrbs['E_FM_F_1'] == 1) & (kyrbs['E_FM_SF_2'] == 2)].shape[0])
print(kyrbs[(kyrbs['E_FM_F_1'] == 9999) & (kyrbs['E_FM_SF_2'] == 9999)].shape[0])

print("\nmother")
print(kyrbs[(kyrbs['E_FM_M_3'] == 3) & (kyrbs['E_FM_SM_4'] == 9999)].shape[0])
print(kyrbs[(kyrbs['E_FM_M_3'] == 9999) & (kyrbs['E_FM_SM_4'] == 4)].shape[0])
print(kyrbs[(kyrbs['E_FM_M_3'] == 3) & (kyrbs['E_FM_SM_4'] == 4)].shape[0])
print(kyrbs[(kyrbs['E_FM_M_3'] == 9999) & (kyrbs['E_FM_SM_4'] == 9999)].shape[0])

father
45394
286
1446
1610

mother
46095
168
1310
1163


In [12]:
display(kyrbs['E_LT_F'].value_counts())
display(kyrbs['E_LT_SF'].value_counts())

E_LT_F
1.0       42343
2.0        4497
9999.0     1896
Name: count, dtype: int64

E_LT_SF
9999.0    47004
2.0        1303
1.0         429
Name: count, dtype: int64

In [13]:
display(kyrbs['E_LT_M'].value_counts()) 
display(kyrbs['E_LT_SM'].value_counts())

E_LT_M
1.0       45588
2.0        1817
9999.0     1331
Name: count, dtype: int64

E_LT_SM
9999.0    47258
2.0        1288
1.0         190
Name: count, dtype: int64

In [14]:
def count_parents(row):
    # 친부, 계부 확인
    father = 1 if row['E_FM_F_1'] == 1 or row['E_FM_SF_2'] == 2 else 0  
    # 친모, 계모 확인
    mother = 1 if row['E_FM_M_3'] == 3 or row['E_FM_SM_4'] == 4 else 0  
    return father + mother  # 부모 수 합산 (최대 2)

kyrbs['countparent'] = kyrbs.apply(count_parents, axis=1)

In [15]:
kyrbs['countparent'].value_counts()

countparent
2    46271
1     2157
0      308
Name: count, dtype: int64

In [16]:
kyrbs['stepparent'] = kyrbs[['E_FM_SF_2', 'E_FM_SM_4']].apply(lambda x: 1 if (x == 2).any() or (x == 4).any() else 0, axis=1)
kyrbs['stepparent'].value_counts()

stepparent
0    46760
1     1976
Name: count, dtype: int64

In [17]:
display(kyrbs['E_FM_OBS_7'].value_counts())
display(kyrbs['E_FM_YBS_8'].value_counts())
display(kyrbs['E_FM_NO_9'].value_counts())

E_FM_OBS_7
9999.0    24534
7.0       24202
Name: count, dtype: int64

E_FM_YBS_8
9999.0    26355
8.0       22381
Name: count, dtype: int64

E_FM_NO_9
9999.0    48603
9.0         133
Name: count, dtype: int64

In [18]:
kyrbs['brosis'] = kyrbs[['E_FM_OBS_7', 'E_FM_YBS_8']].apply(lambda x: 1 if (x == 7).any() or (x == 8).any() else 2, axis=1)
kyrbs['brosis'].value_counts()

brosis
1    41187
2     7549
Name: count, dtype: int64

In [19]:
kyrbs.isna().sum()

OBS            0
mod_d          0
YEAR           0
CITY           0
CTYPE          0
              ..
disturbed      0
violent        0
countparent    0
stepparent     0
brosis         0
Length: 199, dtype: int64

In [20]:
kyrbs.columns

Index(['OBS', 'mod_d', 'YEAR', 'CITY', 'CTYPE', 'CTYPE_SD', 'MH', 'SCHOOL',
       'STYPE', 'STRATA',
       ...
       'E_KRN_F', 'E_BORN_F', 'E_EDU_M', 'E_KRN_M', 'E_BORN_M', 'disturbed',
       'violent', 'countparent', 'stepparent', 'brosis'],
      dtype='object', length=199)

In [21]:
usecols = ['MH', 'M_STR', 'M_SAD', 'M_LON', 'disturbed', 'PR_HT', 'E_S_RCRD', 'M_SUI_CON', 'M_SUI_PLN', 'M_SUI_ATT', 'E_SES', 'countparent', 'stepparent', 'brosis', 'E_RES', 'violent']
kyrbs = kyrbs[usecols]
kyrbs.head(3)

,MH,M_STR,M_SAD,M_LON,disturbed,PR_HT,E_S_RCRD,M_SUI_CON,M_SUI_PLN,M_SUI_ATT,E_SES,countparent,stepparent,brosis,E_RES,violent
0,중학교,4.0,1.0,2.0,1,1.0,1.0,1.0,1.0,1.0,1.0,2,0,1,1.0,1
1,중학교,3.0,2.0,2.0,1,1.0,4.0,1.0,1.0,1.0,2.0,2,0,1,1.0,1
2,중학교,2.0,1.0,3.0,1,1.0,1.0,1.0,1.0,1.0,1.0,2,0,1,1.0,1


In [22]:
kyrbs = kyrbs.dropna()

In [23]:
# 스트레스 척도 변환 (1(많이) ~ 5(전혀) -> 5(많이) ~ 1(전혀))
kyrbs['M_STR'] = 6 - kyrbs['M_STR']

# 건강 상태 척도 변환 (1(건강함) ~ 5(건강 못함) -> 5(건강함) ~ 1(건강 못함))
kyrbs['PR_HT'] = 6 - kyrbs['PR_HT']

# 학업 성적 척도 변환 (1(상) ~ 5(하) -> 5(상) ~ 1(하))
kyrbs['E_S_RCRD'] = 6 - kyrbs['E_S_RCRD']

# 경제 상태 척도 변환 (1(상) ~ 5(하) -> 5(상) ~ 1(하))
kyrbs['E_SES'] = 6 - kyrbs['E_SES']


In [24]:
# 숫자형 변수
numeric_cols = ['M_STR', 'M_LON', 'PR_HT', 'E_S_RCRD', 'E_SES', 'violent']
for col in numeric_cols:
    kyrbs[col] = pd.to_numeric(kyrbs[col], errors='coerce')

# 범주형 변수
categorical_cols = ['M_SAD', 'disturbed', 'M_SUI_CON', 'M_SUI_PLN', 'M_SUI_ATT', 
                    'countparent', 'stepparent', 'brosis', 'E_RES']
for col in categorical_cols:
    kyrbs[col] = kyrbs[col].astype("category")

# # 순서형 변수
# ordinal_cols = ['violent']
# violent_categories = ['never', 'sometimes', 'often']

# for col in ordinal_cols:
#     kyrbs[col] = kyrbs[col].astype(pd.CategoricalDtype(categories=violent_categories, ordered=True))

In [25]:
mid = kyrbs[kyrbs['MH'] == '중학교']
high = kyrbs[kyrbs['MH'] == '고등학교']

len(mid), len(high)

(26323, 22413)

In [26]:
high.dtypes

MH               object
M_STR           float64
M_SAD          category
M_LON           float64
disturbed      category
PR_HT           float64
E_S_RCRD        float64
M_SUI_CON      category
M_SUI_PLN      category
M_SUI_ATT      category
E_SES           float64
countparent    category
stepparent     category
brosis         category
E_RES          category
violent           int64
dtype: object

In [27]:
import pandas as pd
from semopy import Model, Optimizer, calc_stats

In [43]:
from semopy import Model

model_desc = """
    stress =~ M_STR
    mood =~ M_SAD + M_LON + disturbed
    suicide =~ M_SUI_CON + M_SUI_PLN + M_SUI_ATT
    grade =~ E_S_RCRD
    family =~ E_SES + countparent + stepparent
    livingspace =~ E_RES
    
    mood ~ grade + family + livingspace
    stress ~ grade + family + livingspace
    suicide ~ stress + mood
"""

m = Model(model_desc)
print(model_desc)

m.fit(high)

# 구조 모델 결과 확인
stats = calc_stats(m)

print("적합도 지표:\n", stats.T)



    stress =~ M_STR
    mood =~ M_SAD + M_LON + disturbed
    suicide =~ M_SUI_CON + M_SUI_PLN + M_SUI_ATT
    grade =~ E_S_RCRD
    family =~ E_SES + countparent + stepparent
    livingspace =~ E_RES
    
    mood ~ grade + family + livingspace
    stress ~ grade + family + livingspace
    suicide ~ stress + mood

적합도 지표:
                       Value
DoF               43.000000
DoF Baseline      66.000000
chi2            2434.663602
chi2 p-value       0.000000
chi2 Baseline  39379.955598
CFI                0.939165
GFI                0.938175
AGFI               0.905106
NFI                0.938175
TLI                0.906625
RMSEA              0.049817
AIC               69.782745
BIC              350.391620
LogLik             0.108627


In [32]:
estimates = m.inspect()
print(estimates)

                   lval  op                 rval  Estimate  Std. Err  \
0                Stress   ~   Family_Environment  0.040067   0.01462   
1                Stress   ~  Violence_Experience  0.405067  0.000413   
2                Stress   ~               Health -0.255359   0.00701   
3                Stress   ~             Academic -0.060199  0.023395   
4                 E_SES   ~   Family_Environment  1.000000         -   
5           countparent   ~   Family_Environment  0.054367  0.006439   
6            stepparent   ~   Family_Environment -0.010142  0.002104   
7                brosis   ~   Family_Environment -0.004186   0.00374   
8                 E_RES   ~   Family_Environment -0.044808  0.009255   
9               violent   ~  Violence_Experience  1.000000         -   
10                PR_HT   ~               Health  1.000000         -   
11             E_S_RCRD   ~             Academic  1.000000         -   
12                M_STR   ~               Stress  1.000000      

In [33]:
from semopy import semplot, report
semplot(m, './high.png')
report(m, './report')